In [ ]:
import librosa
import numpy as np
from GTZAN import *

In [ ]:
hp = {}
hp['sr'] = 16000
hp['hl'] = 512
hp['nm'] = 229
hp['fm'] = 30.0

In [ ]:
def get_mel(file, hp):
    wav = librosa.load(file, hp['sr'])[0]
    mel = librosa.feature.melspectrogram(
            wav,
            sr=hp['sr'],
            hop_length=hp['hl'],
            fmin=hp['fm'],
            n_mels=hp['nm'],
            htk=False
            ).astype(np.float32)
    mel = mel.T
    return mel

In [ ]:
for s in styles:
    for n in range(100):
        mel = get_mel(filename_audio(s, n), hp)
        np.save(filename_spec(s, n), mel)
        print(filename(s, n))